<a href="https://colab.research.google.com/github/umeshnandargi/Pytorch/blob/main/Student_depression_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/sample_data/student_lifestyle_100k.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/student_lifestyle_100k.csv'

In [ ]:
labels = df.columns[1:10]
labels

Index(['Age', 'Gender', 'Department', 'CGPA', 'Sleep_Duration', 'Study_Hours',
       'Social_Media_Hours', 'Physical_Activity', 'Stress_Level'],
      dtype='object')

In [ ]:
Departments = dict()
for i, dept in enumerate(df.Department.unique()):
  Departments[dept] = float(i)

Departments

{'Science': 0.0,
 'Engineering': 1.0,
 'Medical': 2.0,
 'Arts': 3.0,
 'Business': 4.0}

In [ ]:
df.Gender = df.Gender.replace({'Female':0.0, 'Male':1.0})
df.Department = df.Department.replace(Departments)
df.dtypes

/tmp/ipython-input-96476380.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.Gender = df.Gender.replace({'Female':0.0, 'Male':1.0})
/tmp/ipython-input-96476380.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.Department = df.Department.replace(Departments)


,0
Student_ID,int64
Age,int64
Gender,float64
Department,float64
CGPA,float64
Sleep_Duration,float64
Study_Hours,float64
Social_Media_Hours,float64
Physical_Activity,int64
Stress_Level,int64


In [ ]:
X = df.drop(['Student_ID' ,'Depression'], axis=1).values
y = df.Depression.values

In [ ]:
y = y.astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class Model(nn.Module):
  # Input Layer   -> 9 (dropping studentID)
  # Hidden Layer1 -> 5
  # Hidden Layer2 -> 5
  # Output        -> 2 (True/False)

  def __init__(self, inp, h1, h2, out) -> None:
    super().__init__()

    self.fc1 = nn.Linear(inp,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2, out)

  def forward(self, x): # shape of x = (batch_size, inp)
    x = F.relu(self.fc1(x)) # xnew​=x⋅WT+b -> shape of x changes -> shape(x) = (batch_size, h1)
    x = F.relu(self.fc2(x)) # shape(x) = (batch_size, h2)
    x = self.out(x) # shape(x) = (batch_size, out)

    return x

In [ ]:
torch.manual_seed(43)
model = Model(9,12,12,2)

In [ ]:
lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model.eval()

Model(
  (fc1): Linear(in_features=9, out_features=12, bias=True)
  (fc2): Linear(in_features=12, out_features=12, bias=True)
  (out): Linear(in_features=12, out_features=2, bias=True)
)

In [ ]:
epochs = 1000
batch = 2
losses = []

for i in range(batch):
  for j in range(epochs):
    y_pred = model.forward(X_train)
    loss = lossfn(y_pred, y_train) # Corrected argument order
    losses.append(loss)

    if j%1000 ==0:
      print(f'Batch : {i} and loss: {loss:.5f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Batch : 0 and loss: 5.00203
Batch : 1 and loss: 0.30342


In [ ]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    # print(f'{i+1}.) {str(y_val) } \t {y_test[i]} \t {y_val.argmax().item()}')


    if y_val.argmax().item() == y_test[i]:
      correct +=1
    if y_val.argmax().item() == y_test[i] and y_test[i] == 1:
      print(i)

# print(f'{correct} Correct preds!!')
print(f'{correct/len(y_test)}')

401
1213
1322
3234
3705
4950
6316
10411
11097
13630
13969
15736
16413
16682
16983
17400
17778
18910
0.9008


In [ ]:
labels
age = 26.0
gender = 1.0
department = Departments['Engineering']
cgpa = 1.2
sleep_duration = 3.0
study_hours = 1.0
social_media_hours = 7.0
physical_activity = 10
stress_level = 9
trial = torch.tensor([age, gender, department, cgpa, sleep_duration, study_hours, social_media_hours, physical_activity,
                       stress_level])
trial

trial, X_test[0]

NameError: name 'labels' is not defined

In [ ]:
with torch.no_grad():
  y_eval = model.forward(trial)

print(y_val.argmax().item())

0


In [ ]:
index = 401
with torch.no_grad():
  y_eval = model.forward(X_test[index])

print(y_eval.argmax().item(), y_test[index])
X_test[index]

NameError: name 'X_test' is not defined

In [ ]:
torch.save(model.state_dict(), 'depression_model')

In [ ]:
saved_model = Model(9,12,12,2)
saved_model.load_state_dict(torch.load('depression_model'))

<All keys matched successfully>

In [ ]:
epochs = 1000
batch = 5
losses = []

for i in range(batch):
  for j in range(epochs):
    y_pred = saved_model.forward(X_train)
    loss = lossfn(y_pred, y_train) # Corrected argument order
    losses.append(loss)

    if j%1000 ==0:
      print(f'Batch : {i} and loss: {loss:.5f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Batch : 0 and loss: 0.30322
Batch : 1 and loss: 0.30322
